<a href="https://colab.research.google.com/github/AvinaashAnandK/DeepLearningJourney/blob/main/%5BDeep_Learning_Assignments%5D_%5BCS198%5D_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using the MNIST dataset, achieve 90% accuracy with a fully connected network.
1. Epochs: 1
2. Use Nesterov momentum in optim.SGD

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [2]:
# @title Util to display a sample image
def display_input_image(input,input_type="np"):
  if type not in ["np","tensor"]:
    img_as_np = plt.imread(input)
    processed_type = "np"
  else:
    processed_type = input_type

  print("Displaying image:")

  if processed_type == "np":
    plt.imshow(img_as_np)
  else:
    plt.imshow(input.permute(1,2,0))

  plt.show()

In [3]:
# @title Downloading the dataset - MNIST

mnist_train = datasets.MNIST(root = "./data", train = True, download = True)
mnist_test = datasets.MNIST(root = "./data", train = False, download = True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:02<00:00, 4206853.15it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 65690.10it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:06<00:00, 245979.30it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 10269826.83it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [4]:
# Accessing dataset information
print(f"Shape of the dataset: {mnist_train.data.shape}")
print(f"Shape of the labels: {mnist_train.targets.shape}")
print(f"Classes: {mnist_train.classes}")

Shape of the dataset: torch.Size([60000, 28, 28])
Shape of the labels: torch.Size([60000])
Classes: ['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four', '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']


In [5]:
# Creating loaders for train, validation (for hyperparam training) and testing
transform = T.Compose([
    T.ToTensor(),
    T.Normalize((0.5, ), (0.5, ))
])

mnist_train = datasets.MNIST(root = "./data", train = True, download = True, transform = transform)
mnist_test = datasets.MNIST(root = "./data", train = False, download = True, transform = transform)

In [6]:
model_params = {
    "hidden_layer_dim": 4000,
    "num_layers": 5,
    "classification_labels": 10,
    "epochs": 1,
    "learning_rate": 1e-2,
    "batch_size": 32,
    "test_val_split": 0.1,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "train_every": 100,
    "nesterov": True,
    "momentum": 0.9
}

model_params['input_dims'] = np.prod(mnist_train.data.shape[1:])

# Change model_params
batch_size = 64
model_params['batch_size'] = batch_size

In [7]:
DATASET_SIZE = mnist_train.data.shape[0]
NUM_TRAIN = int(DATASET_SIZE *(1-model_params['test_val_split']))
NUM_VAL = DATASET_SIZE - NUM_TRAIN
print(f"MNIST Dataset has {DATASET_SIZE} images for training, Training Set Length: {NUM_TRAIN} after validation set of {NUM_VAL} images.")

MNIST Dataset has 60000 images for training, Training Set Length: 54000 after validation set of 6000 images.


In [8]:
train_set, val_set = torch.utils.data.random_split(mnist_train, [NUM_TRAIN, NUM_VAL])
train = DataLoader(train_set, batch_size = model_params['batch_size'], shuffle = True)
val = DataLoader(val_set, batch_size = model_params['batch_size'], shuffle = True)
test = DataLoader(mnist_test, batch_size = model_params['batch_size'], shuffle = True)

In [9]:
def flatten(x):
  batch = x.shape[0]
  return x.view(batch, -1)

class FiveLayerFC(nn.Module):
  def __init__(self, params):
    super().__init__()
    self.num_layers = params['num_layers']
    self.output_size = params['classification_labels']
    self.hidden_size = params['hidden_layer_dim']
    self.input_size = params['input_dims']

    self.layers = nn.ModuleList()

    self.layers.append(nn.Linear(self.input_size, self.hidden_size))
    self.layers.append(nn.ReLU())

    for _ in range(self.num_layers-1):
      self.layers.append(nn.Linear(self.hidden_size,self.hidden_size))
      self.layers.append(nn.ReLU())

    self.layers.append(nn.Linear(self.hidden_size, self.output_size))

  def forward(self, x):
    x = flatten(x)
    for layer in self.layers:
      x = layer(x)
    return x

def check_accuracy(loader, model, model_params):
  device = model_params['device']
  pred_correct, total_count = 0, 0

  model.eval()
  with torch.no_grad():
    for batch, (inputs, labels) in tqdm(enumerate(loader), total=len(loader), desc="Checking Accuracy"):
      inputs, labels = inputs.to(device), labels.to(device)
      preds = model(inputs)
      _, pred_labels = preds.max(1)
      pred_correct += (pred_labels == labels).sum()
      total_count += preds.shape[0]

  accuracy = float(pred_correct)/float(total_count)
  print(f"Accuracy: {accuracy*100}%")

  return accuracy

def train_model(loader_train, loader_val, model, optimizer, model_params):
  device = model_params['device']
  epochs = model_params['epochs']
  train_every = model_params['train_every']

  lossfn = nn.CrossEntropyLoss()

  train_acc, val_acc = [], []

  model.to(device)


  for epoch in tqdm(range(epochs),desc="Epochs"):
    model.train()
    for batch, (inputs, labels) in tqdm(enumerate(loader_train),total=len(loader_train),desc=f"Batches in epoch {epoch}"):
      inputs, labels = inputs.to(device), labels.to(device)
      preds = model(inputs)

      loss = lossfn(preds, labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      if batch % train_every == 0:
        train_acc.append(check_accuracy(loader_train, model, model_params))

    val_acc.append(check_accuracy(loader_val, model, model_params))
  return train_acc, val_acc

In [20]:
model_params['learning_rate']

0.01

In [28]:
if torch.cuda.is_available():
  model_params['device'] = torch.device("cuda")
else:
  model_params['device'] = torch.device("cpu")

print(f"Using device: {model_params['device']}")
model = FiveLayerFC(model_params)

optimizer = optim.SGD(model.parameters(), lr=model_params['learning_rate'], momentum=model_params['momentum'])
# optimizer = optim.Adam(model.parameters(), lr=0.001)

Using device: cuda


In [29]:
train_acc, val_acc = train_model(train, val, model, optimizer, model_params)

Batches in epoch 0:   0%|          | 0/844 [00:00<?, ?it/s]

Checking Accuracy:   0%|          | 0/844 [00:00<?, ?it/s]

Checking Accuracy:   1%|          | 8/844 [00:00<00:11, 73.52it/s]

Checking Accuracy:   2%|▏         | 16/844 [00:00<00:11, 69.50it/s]

Checking Accuracy:   3%|▎         | 23/844 [00:00<00:12, 66.34it/s]

Checking Accuracy:   4%|▎         | 30/844 [00:00<00:12, 64.18it/s]

Checking Accuracy:   4%|▍         | 37/844 [00:00<00:13, 60.46it/s]

Checking Accuracy:   5%|▌         | 44/844 [00:00<00:12, 62.40it/s]

Checking Accuracy:   6%|▌         | 51/844 [00:00<00:12, 64.07it/s]

Checking Accuracy:   7%|▋         | 58/844 [00:00<00:11, 65.62it/s]

Checking Accuracy:   8%|▊         | 65/844 [00:00<00:11, 65.99it/s]

Checking Accuracy:   9%|▊         | 72/844 [00:01<00:11, 66.94it/s]

Checking Accuracy:   9%|▉         | 79/844 [00:01<00:11, 66.85it/s]

Checking Accuracy:  10%|█         | 86/844 [00:01<00:11, 66.91it/s]

Checking Accuracy:  11%|█         | 93/844 [00:01<00

Accuracy: 8.88888888888889%



Batches in epoch 0:  12%|█▏        | 100/844 [00:15<00:14, 50.55it/s]

Checking Accuracy:   0%|          | 0/844 [00:00<?, ?it/s]

Checking Accuracy:   1%|          | 8/844 [00:00<00:11, 72.00it/s]

Checking Accuracy:   2%|▏         | 16/844 [00:00<00:12, 67.36it/s]

Checking Accuracy:   3%|▎         | 23/844 [00:00<00:12, 68.38it/s]

Checking Accuracy:   4%|▎         | 31/844 [00:00<00:11, 68.58it/s]

Checking Accuracy:   5%|▍         | 39/844 [00:00<00:11, 69.85it/s]

Checking Accuracy:   6%|▌         | 47/844 [00:00<00:11, 69.47it/s]

Checking Accuracy:   6%|▋         | 54/844 [00:00<00:12, 64.37it/s]

Checking Accuracy:   7%|▋         | 62/844 [00:00<00:11, 66.35it/s]

Checking Accuracy:   8%|▊         | 69/844 [00:01<00:11, 66.56it/s]

Checking Accuracy:   9%|▉         | 77/844 [00:01<00:11, 68.86it/s]

Checking Accuracy:  10%|▉         | 84/844 [00:01<00:11, 67.48it/s]

Checking Accuracy:  11%|█         | 92/844 [00:01<00:10, 68.57it/s]

Checking Accuracy:  12%|█▏        | 100/8

Accuracy: 59.57222222222222%



Batches in epoch 0:  24%|██▎       | 200/844 [00:31<00:15, 40.34it/s]

Checking Accuracy:   0%|          | 0/844 [00:00<?, ?it/s]

Checking Accuracy:   1%|          | 5/844 [00:00<00:17, 48.71it/s]

Checking Accuracy:   1%|          | 10/844 [00:00<00:18, 45.95it/s]

Checking Accuracy:   2%|▏         | 15/844 [00:00<00:18, 44.32it/s]

Checking Accuracy:   2%|▏         | 20/844 [00:00<00:18, 44.49it/s]

Checking Accuracy:   3%|▎         | 25/844 [00:00<00:18, 44.84it/s]

Checking Accuracy:   4%|▎         | 30/844 [00:00<00:24, 33.86it/s]

Checking Accuracy:   4%|▍         | 34/844 [00:01<00:40, 20.02it/s]

Checking Accuracy:   5%|▍         | 38/844 [00:01<00:35, 23.00it/s]

Checking Accuracy:   5%|▌         | 45/844 [00:01<00:25, 31.56it/s]

Checking Accuracy:   6%|▋         | 53/844 [00:01<00:19, 41.06it/s]

Checking Accuracy:   7%|▋         | 60/844 [00:01<00:16, 47.15it/s]

Checking Accuracy:   8%|▊         | 66/844 [00:01<00:18, 41.89it/s]

Checking Accuracy:   8%|▊         | 71/84

...

Accuracy: 93.73518518518519%



Batches in epoch 0:  95%|█████████▍| 798/844 [02:09<00:00, 49.87it/s]

Checking Accuracy:   0%|          | 0/844 [00:00<?, ?it/s]

Checking Accuracy:   1%|          | 7/844 [00:00<00:13, 64.08it/s]

Checking Accuracy:   2%|▏         | 14/844 [00:00<00:12, 66.98it/s]

Checking Accuracy:   3%|▎         | 22/844 [00:00<00:11, 70.05it/s]

Checking Accuracy:   4%|▎         | 30/844 [00:00<00:11, 69.69it/s]

Checking Accuracy:   4%|▍         | 37/844 [00:00<00:12, 65.41it/s]

Checking Accuracy:   5%|▌         | 44/844 [00:00<00:12, 63.27it/s]

Checking Accuracy:   6%|▌         | 51/844 [00:00<00:12, 63.24it/s]

Checking Accuracy:   7%|▋         | 58/844 [00:00<00:12, 65.06it/s]

Checking Accuracy:   8%|▊         | 65/844 [00:00<00:11, 66.20it/s]

Checking Accuracy:   9%|▊         | 72/844 [00:01<00:11, 64.44it/s]

Checking Accuracy:   9%|▉         | 79/844 [00:01<00:11, 64.16it/s]

Checking Accuracy:  10%|█         | 86/844 [00:01<00:11, 65.49it/s]

Checking Accuracy:  11%|█         | 93/84

Accuracy: 93.85555555555555%



Batches in epoch 0: 100%|██████████| 844/844 [02:25<00:00,  5.80it/s]

Epochs: 100%|██████████| 1/1 [02:27<00:00, 147.45s/it]

Accuracy: 94.38333333333333%


In [30]:
model_params

{'hidden_layer_dim': 4000,
 'num_layers': 5,
 'classification_labels': 10,
 'epochs': 1,
 'learning_rate': 0.01,
 'batch_size': 64,
 'test_val_split': 0.1,
 'device': device(type='cuda'),
 'train_every': 100,
 'nesterov': True,
 'momentum': 0.9,
 'input_dims': 784}

In [31]:
train_acc

[0.08888888888888889,
 0.5957222222222223,
 0.837462962962963,
 0.8916481481481482,
 0.8914814814814814,
 0.9202222222222223,
 0.9349074074074074,
 0.9373518518518519,
 0.9385555555555556]

In [32]:
val_acc

[0.9438333333333333]